<div style="font-size:18pt; padding-top:20px; text-align:center">СЕМИНАР. <b>Кластеризация текстовых документов и </b> <span style="font-weight:bold; color:green">Sklearn/NLTK/Gensim</span></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin.study@yandex.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Загрузка исходных данных</a></li>
        <li><a href="#2">Лемматизация, стемминг и POS</a></li>
        <li><a href="#3">Latent Dirichlet Allocation (LDA)</a>
        <li><a href="#4">Визуализация с WordCloud</a></li>
        <li><a href="#5">PCA преобразование для текстовых данных</a></li>
        <li><a href="#6">t-SNE преобразование для текстовых данных</a></li>
        <li><a href="#7">Источники</a>
        </li>
    </ol>
</div>

<p>Подключение библиотек</p>

In [ ]:
import numpy as np

In [ ]:
from gensim import corpora, matutils, models
from sklearn.datasets import fetch_20newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Загрузка исходных данных</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p><b>Исходные данные</b></p>

In [ ]:
topics = ["sci.space", "soc.religion.christian", "rec.sport.baseball", "comp.sys.mac.hardware"]

In [ ]:
newsgroups_train = fetch_20newsgroups(subset="train", data_home="data", 
                                      categories=topics, remove=("headers", "footers", "quotes"))

In [ ]:
newsgroups_test = fetch_20newsgroups(subset="test", data_home="data", 
                                      categories=topics, remove=("headers", "footers", "quotes"))

In [ ]:
newsgroups_train.target_names

<p>Пример документа</p>

In [ ]:
documents = newsgroups_train.data
topics_true = newsgroups_train.target
documents[0]

<p>Количество документов в каждой категории</p>

In [ ]:
list(zip(newsgroups_train.target_names, np.bincount(topics_true)))

<p>Словарь категорий/тем</p>

In [ ]:
d_init_topics = {i:newsgroups_train.target_names[i] for i in range(len(newsgroups_train.target_names))}
d_init_topics

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Лемматизация, стемминг и POS</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
import nltk

<p>Для загрузки моделей, данных и пр.</p>

In [ ]:
nltk.download()

<p><b>Разделение текста на токены</b></p>

In [ ]:
from nltk.tokenize import word_tokenize, RegexpTokenizer

In [ ]:
tokenizer = RegexpTokenizer(r"\w+")

In [ ]:
doc_tokens = tokenizer.tokenize(documents[0])
doc_tokens[:20]

In [ ]:
nltk.download('punkt_tab')
doc_tokens = word_tokenize(documents[0])
doc_tokens[:20]

<p><b>Стоп-слова</b></p>

In [ ]:
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words("english") 
stop_words

<p><b>Лемматизация</b></p>

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
lemmatizer.lemmatize("matrices", "n")

In [ ]:
lemmatizer.lemmatize("everybody", "n")

In [ ]:
lemmatizer.lemmatize("easier", "a")

In [ ]:
lemmatizer.lemmatize("trying", "v")

In [ ]:
lemmatizer.lemmatize("tried", "v")

<p><b>Cтемминг</b></p>

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
stemmer = PorterStemmer()

In [ ]:
stemmer.stem("everybody")

In [ ]:
stemmer.stem("trying")

In [ ]:
stemmer.stem("tried")

In [ ]:
stemmer.stem("try")

<p>Русский язык</p>

In [ ]:
from nltk.stem.snowball import RussianStemmer

In [ ]:
rus_stemmer = RussianStemmer()

In [ ]:
rus_stemmer.stem("учиться")

In [ ]:
rus_stemmer.stem("заучиться")

<p>Пример</p>

In [ ]:
chars = "-[],.<>()={}'\"?!`~/|+*&^%$#@;: \\_"

In [ ]:
docs_tokens = list()

for doc in documents:
    doc = doc.lower()
    stemmed_tokens = [stemmer.stem(el) for el in word_tokenize(doc) 
                      if len(el) > 1 and not any(ch.isdigit() or ch in chars for ch in el) 
                      and el not in stop_words]
    docs_tokens.append(stemmed_tokens)
    
docs_tokens[:2]

<p><b>Определение части речи (POS)</b></p>

In [ ]:
from nltk.tag import pos_tag
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('averaged_perceptron_tagger_rus')

In [ ]:
pos_tag(["study"])

In [ ]:
pos_tag(["I","study"])

<a href="http://www.winwaed.com/blog/2011/11/08/part-of-speech-tags/">Part of Speech Tags</a>

In [ ]:
nltk.pos_tag(documents[0].split())

<p>Русский язык</p>

In [ ]:
pos_tag(["Я"], lang="rus")

In [ ]:
pos_tag(["учиться"], lang="rus")

In [ ]:
pos_tag(["машина"], lang="rus")

<a href="http://www.ruscorpora.ru/corpora-morph.html">Морфология</a>

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">3. Latent Dirichlet Allocation (LDA)</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p><b>Тематическое моделирование с LDA</b> (кластеризация текстовых документов)</p>

<p>Векторизация TF-IDF</p>

In [ ]:
def mytokenizer(x):
    for el in x.split():
        el = el.strip(chars)
        if len(el) > 1 and not any(ch.isdigit() or ch in chars for ch in el) \
        and el not in stop_words and nltk.pos_tag([el])[0][1]=="NN":
            yield el #stemmer.stem(el)

# TODO: extract POS from sentence instead of single word

In [ ]:
vectorizer = TfidfVectorizer(min_df=20,
                             max_df=0.1, tokenizer=mytokenizer,
                             lowercase=True) #, max_features=1000)

In [ ]:
X = vectorizer.fit_transform(documents)

<p>Словарь "слово-id"</p>

In [ ]:
vectorizer.vocabulary_

<p>Элементы словаря</p>

In [ ]:
tokens = vectorizer.get_feature_names_out()
tokens[:5]

<p>Создания словаря для обратного преобразования id в слово</p>

In [ ]:
id2token = dict()
for tid in range(len(tokens)):
    id2token[tid] = tokens[tid]

<p>Преобразование документов в список кортежи (id, tf-idf)</p>

In [ ]:
corpus_new = list()
for el in X:
    corpus_new.append(list(zip(el.indices, el.data)))

In [ ]:
corpus_new[1][:5]

<p>Кластеризация</p>

In [ ]:
lda = models.ldamodel.LdaModel(corpus=corpus_new, num_topics=4, id2word=id2token, passes=10, random_state=1234)

<p>Вывод полученных тем</p>

In [ ]:
lda.print_topics(4)

<p>Топ 100 слов темы с индексом 2</p>

In [ ]:
top_100_id = lda.get_topic_terms(2, topn=100)
top_100_id[:4]

In [ ]:
top_100_word = lda.show_topic(topicid=2, topn=100)
top_100_word[:4]

<p>Предсказание тем по наибольшей вероятности</p>

In [ ]:
topics_pred = np.array([np.argmax(np.array(el)[:,1]) for el in lda.get_document_topics(corpus_new)])
topics_pred[:5]

<p><b>Простой тест</b></p>

<p>Документ из тестового подмножества</p>

In [ ]:
x_test, y_target = newsgroups_test.data[1], newsgroups_test.target[1]
x_test

<p>Действительная тема</p>

In [ ]:
d_init_topics[y_target]

<p>Преобразование тестового документа в векторы вид</p>

In [ ]:
corpus_test = list()
for el in mytokenizer(x_test):
    el = el.lower()
    if el in vectorizer.vocabulary_:
        corpus_test.append((vectorizer.vocabulary_[el], 1.0))

corpus_test

In [ ]:
for w in corpus_test:
    print(id2token[w[0]])

<p>Предсказание темы с LDA</p>

In [ ]:
pr_topics = lda.get_document_topics(corpus_test)
pr_topics

<p>Индекс темы с наибольшей вероятностью</p>

In [ ]:
max_pr_indx = np.argmax(np.array(pr_topics)[:,1])
max_pr_indx

<p>Первые топ-20 слов из темы с найденным индексом</p>

In [ ]:
lda.show_topic(topicid=max_pr_indx, topn=20)

<p>Подходит тема?</p>

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">4. Визуализация с WordCloud</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
from wordcloud import WordCloud

<p><b>Визуализация слов одной темы</b></p>

<p>Топ 100 слов из темы с индексом 2</p>

In [ ]:
top100 = lda.get_topic_terms(2, topn=100)
top100[:4]

<p>Словарь с вероятностями топ n слов</p>

In [ ]:
dict_top = {id2token[id]:pr for id, pr in top100}
dict_top

<p>Создание WordCloud</p>

In [ ]:
wc = WordCloud(background_color="white", 
               max_words=100, 
               width=1600,
               height=1200).generate_from_frequencies(dict_top)

<p>Отображение</p>

In [ ]:
plt.figure(figsize=[10,10])
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

<a name="5"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">5. PCA преобразование для текстовых данных</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
from sklearn.decomposition import PCA, TruncatedSVD

<p>Из gensim в numpy разреженную матрицу</p>

In [ ]:
x_sparse = matutils.corpus2csc(corpus_new).T

<p>Уменьшение размерности</p>

In [ ]:
m_pca = PCA(n_components=2)
x_pca = m_pca.fit_transform(x_sparse.toarray())

<p>График</p>

In [ ]:
plt.figure(figsize=[15,6])

plt.subplot(1,2,1)
plt.title("True Topics")
plt.scatter(x_pca[:, 0], x_pca[:, 1], c=topics_true)
plt.xlabel("PCA_Component_1")
plt.ylabel("PCA_Component_2")
plt.grid(True)

plt.subplot(1,2,2)
plt.title("Predicted Topics")
plt.scatter(x_pca[:, 0], x_pca[:, 1], c=topics_pred)
plt.xlabel("PCA_Component_1")
plt.ylabel("PCA_Component_2")
plt.grid(True)

plt.show()

<p>Для каждой темы отдельно</p>

In [ ]:
plt.figure(figsize=[12,12])

topic_number = len(d_init_topics)

for i in range(len(d_init_topics)):
    plt.subplot(topic_number//2+1, 2, i+1)
    plt.title("Topic " + str(i))
    topic_indxs = np.where(topics_pred==i)
    plt.scatter(x_pca[topic_indxs, 0], 
            x_pca[topic_indxs, 1], 
            c="grey")#np.random.rand(3,1))
    plt.xlabel("PCA_Component_1")
    plt.ylabel("PCA_Component_2")
    plt.grid(True)
    
plt.tight_layout()
plt.show()

<p><b>Из Truncated SVD в PCA</b></p>

<p>Уменьшение размерности</p>

In [ ]:
x_tsvd = TruncatedSVD(n_components=2).fit_transform(x_sparse)

<p>Отображение</p>

In [ ]:
plt.figure(figsize=[5,5])

plt.scatter(x_tsvd[:, 0], x_tsvd[:, 1], c=topics_pred, zorder=2)
plt.xlabel("TSVD_Component_1")
plt.ylabel("TSVD_Component_2")
plt.grid(True)

plt.show()

<p>Центровка</p>

In [ ]:
x_centered = x_sparse - x_sparse.mean(axis=0)
x_centered

<p>Преобразование</p>

In [ ]:
x_centered = np.asarray(x_centered)
x_tsvd_centered = TruncatedSVD(n_components=2).fit_transform(x_centered)

<p>График</p>

In [ ]:
plt.figure(figsize=[8,8])

plt.title("Predicted Topics")
plt.scatter(x_tsvd_centered[:, 0], x_tsvd_centered[:, 1], c=topics_pred, zorder=2)
plt.xlabel("TSVD_Component_1")
plt.ylabel("TSVD_Component_2")
plt.grid(True)

plt.show()

<a name="6"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">6. t-SNE преобразование для текстовых данных</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
from sklearn.manifold import TSNE

<p>Преобразование</p>

In [ ]:
x_tsne_pca = TSNE(n_components=2, 
                  learning_rate=500, init="pca", 
                  n_iter=1000,
                  random_state=1234).fit_transform(x_sparse.toarray())

<p>Графики</p>

In [ ]:
plt.figure(figsize=[15,6])

plt.subplot(1,2,1)
plt.title("True Topics")
plt.scatter(x_tsne_pca[:, 0], x_tsne_pca[:, 1], c=topics_true, zorder=2)
plt.xlabel("TSNE_Component_1")
plt.ylabel("TSNE_Component_2")
plt.grid(True)

plt.subplot(1,2,2)
plt.title("Predicted Topics")
plt.scatter(x_tsne_pca[:, 0], x_tsne_pca[:, 1], c=topics_pred, zorder=2)
plt.xlabel("TSNE_Component_1")
plt.ylabel("TSNE_Component_2")
plt.grid(True)

plt.show()

In [ ]:
plt.figure(figsize=[12,12])

topic_number = len(d_init_topics)

for i in range(len(d_init_topics)):
    plt.subplot(topic_number//2+1, 2, i+1)
    plt.title("Topic " + str(i))
    topic_indxs = np.where(topics_pred==i)
    plt.scatter(x_tsne_pca[topic_indxs, 0], 
            x_tsne_pca[topic_indxs, 1], 
            c="grey")#np.random.rand(3,1))
    plt.xlabel("PCA_Component_1")
    plt.ylabel("PCA_Component_2")
    plt.grid(True)
    
plt.tight_layout()
plt.show()

<p>Вариант с предварительным уменьшением размерности с TSVD</p>

In [ ]:
x_ttsvd = TruncatedSVD(n_components=20).fit_transform(x_sparse)

In [ ]:
x_tsne = TSNE(n_components=2, 
              learning_rate=500,
              perplexity=50,
              random_state=1234, method="exact", 
              n_iter=1000).fit_transform(x_ttsvd)

In [ ]:
plt.figure(figsize=[15,6])

plt.subplot(1,2,1)
plt.title("True Topics")
plt.scatter(x_tsne[:, 0], x_tsne[:, 1], c=topics_true, zorder=2)
plt.xlabel("TSNE_Component_1")
plt.ylabel("TSNE_Component_2")
plt.grid(True)

plt.subplot(1,2,2)
plt.title("Predicted Topics")
plt.scatter(x_tsne[:, 0], x_tsne[:, 1], c=topics_pred, zorder=2)
plt.xlabel("TSNE_Component_1")
plt.ylabel("TSNE_Component_2")
plt.grid(True)

plt.show()

In [ ]:
plt.figure(figsize=[12,12])

topic_number = len(d_init_topics)

for i in range(len(d_init_topics)):
    plt.subplot(topic_number//2+1, 2, i+1)
    plt.title("Topic " + str(i))
    topic_indxs = np.where(topics_pred==i)
    plt.scatter(x_tsne[topic_indxs, 0], 
            x_tsne[topic_indxs, 1], 
            c="grey")#np.random.rand(3,1))
    plt.xlabel("PCA_Component_1")
    plt.ylabel("PCA_Component_2")
    plt.grid(True)
    
plt.tight_layout()
plt.show()

In [ ]:
# TODO: remove items with high uncertainty

<a name="7"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">7. Источники</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
http://qwone.com/~jason/20Newsgroups/
http://scikit-learn.org/stable/datasets/twenty_newsgroups.html

In [ ]:
https://radimrehurek.com/gensim/models/ldamodel.html
https://amueller.github.io/word_cloud/auto_examples/simple.html
http://www.peculiarparity.com/using-gensim-with-andreas-muellers-word-cloud/
http://alexanderfabisch.github.io/t-sne-in-scikit-learn.html